In [ ]:
import os
import pickle
from tqdm import tqdm
from shapely.geometry import Point, LineString
from rtree import index
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import utils.projections as projections

standard_colors = list(mcolors.BASE_COLORS.keys())

In [ ]:
filename = os.path.join("data", 'all_trail_segments.pickle')

with open(filename, 'rb') as f:
    hiking_trails = pickle.load(f)

In [ ]:
filename = os.path.join("data", 'all_lakes.pickle')

with open(filename, 'rb') as f:
    lakes = pickle.load(f)

In [ ]:
# Create an R-tree index
trail_idx = index.Index()

# Insert lines into the R-tree index with their bounding boxes
for i, (line, _) in enumerate(hiking_trails):
    bbox = line.bounds
    trail_idx.insert(i, bbox)

# Function to find the polygon a line is in (O(log(N)) time complexity)
def find_neighbors_for_trail(l, line):
    connected = []
    for i in trail_idx.intersection(line.bounds):
        if i != l:
            # TODO: Consider if need to loosen the intersection if trails don't exactly meet.
            intersection = hiking_trails[i][0].intersection(line)
            if not intersection.is_empty:
                connected.append(i)
    return connected

In [ ]:
trail_subset = hiking_trails
graph = []
for l, (line, _) in tqdm(enumerate(trail_subset)):
    connections = find_neighbors_for_trail(l, line)
    graph.append(connections)

In [ ]:
def print_trail(trail, neighbors):
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))

    rendered_trail = projections.project_line_to_meters(trail[0])

    rendered_neighbors = []
    for n in neighbors:
        rendered_neighbor = projections.project_line_to_meters(hiking_trails[n][0])
        rendered_neighbors.append(rendered_neighbor)

    x, y = rendered_trail.xy
    ax.plot(x, y, label='Trail', color=standard_colors[0])

    for i, n in enumerate(rendered_neighbors):
        x, y = n.xy
        ax.plot(x, y, label='Neighbor {}'.format(i), color=standard_colors[(i+1) % len(standard_colors)])

     # Set labels and legend
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title("Trail {} w/ neighbors".format(trail[1]['name']))
    ax.legend()

    # Set equal aspect ratio
    plt.axis('equal')

    # Show the plot
    plt.show()

In [ ]:
LIMIT = 15
count = 0
for i in range(len(trail_subset)):
    trail = trail_subset[i]

    neighbors = graph[i]
    print_trail(trail, neighbors)

    count += 1
    if count == LIMIT:
        break

In [ ]:
# Searchable attributes of generated trail:
# - D_min <= Total distance <= D_max
# - E_min <= [later] Elevation delta <= E_max
# - Near lake? (K, 1 = yes, 0 = no)
# - Is loop? (P, 1 = yes, 0 = no)

# Graph
# where node is point between trails
# and edge is trail segment
# with following features in attribute vector, A:
# - Distance (A1)
# - Elevation change (A2)
# - Near lake? (A3, 1 = yes, 0 = no)

# Perform DFS and only accept paths with segments, S, where:
# - D_min <= Sum_S[S(A1)] <= D_max
# - E_min <= Sum_S[S(A2)] <= E_max
# - Sum_S[S(A3)] == 1 if K == 1
# - S[0] == S[-1] if P == 1